In [11]:
import pandas as pd
import requests
from time import sleep

# -----------------------------
# 1️⃣ Charger le fichier DVF
# -----------------------------
df = pd.read_csv("ValeursFoncieres.txt", sep="|", header=0, dtype=str, low_memory=False)

cols = ["No voie", "Type de voie", "Voie", "Code postal", "Commune", "Type local", "Surface terrain", "Surface reelle bati"]
df_adresse = df[cols].copy()  # copie pour éviter le warning

# -----------------------------
# 2️⃣ Créer une colonne adresse_txt
# -----------------------------
df_adresse.loc[:, "adresse_txt"] = (
    df_adresse[["No voie", "Type de voie", "Voie"]]
    .fillna("")
    .agg(" ".join, axis=1)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    + ", " +
    df_adresse["Code postal"].fillna("") + " " +
    df_adresse["Commune"].fillna("")
)



In [20]:
# -----------------------------
# 3️⃣ Fonction de géocodage via Nominatim
# -----------------------------
def geocode_osm(adresse):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": adresse,
        "format": "json",
        "addressdetails": 0,
        "limit": 1
    }
    try:
        response = requests.get(url, params=params, headers={"User-Agent": "dvf_geocoding"})
        data = response.json()
        sleep(0.1)  # obligatoire pour ne pas se faire bloquer
        if data:
            return float(data[0]["lat"]), float(data[0]["lon"])
    except:
        pass
    return None, None

# -----------------------------
# 4️⃣ Géocoder juste les 5 premières lignes
# -----------------------------
# Supprimer les doublons dans la colonne adresse_txt
df_adresse_unique = df_adresse.drop_duplicates(subset=["adresse_txt"]).copy()

df_head = df_adresse_unique.head(300).copy()
df_head[["latitude", "longitude"]] = df_head["adresse_txt"].apply(lambda x: pd.Series(geocode_osm(x)))



In [21]:

# -----------------------------
# 5️⃣ Afficher le résultat
# -----------------------------
df_head_clean = df_head.dropna(subset=["latitude", "longitude"])

print(df_head_clean[["adresse_txt", "latitude", "longitude", "Type local", "Surface terrain", "Surface reelle bati"]])

                                         adresse_txt   latitude  longitude  \
0                                FARGES, 1550 FARGES  45.040350   3.459535   
1              454 RUE DE LA REPUBLIQUE, 1550 FARGES  46.172067   5.908144   
5             66 RUE DE LA FRUITIERE, 1200 MONTANGES  46.164577   5.801754   
6                       PETIT CORGENON, 1310 BUELLAS  46.205148   5.156260   
7            459 CHE DU PETIT CORGENON, 1310 BUELLAS  46.206154   5.154267   
..                                               ...        ...        ...   
608         9001 RUE PASTEUR, 1500 AMBERIEU-EN-BUGEY  45.956555   5.345910   
611        384 RUE DES RIVES DE L AIN, 1160 VARAMBON  46.042638   5.317763   
615                      PLATET BERLIET, 1230 CHALEY  45.951778   5.529012   
617  85 RUE ALEXANDRE BERARD, 1500 AMBERIEU-EN-BUGEY  45.962670   5.356638   
620        26 RUE DE MONTHOLON, 1000 BOURG-EN-BRESSE  46.198689   5.210441   

      Type local Surface terrain Surface reelle bati  
0       

In [37]:
import folium

# -----------------------------
# 1️⃣ Définir le centre de la carte
# Ici, on prend la moyenne des coordonnées pour centrer
# -----------------------------
lat_mean = df_head_clean["latitude"].mean()
lon_mean = df_head_clean["longitude"].mean()

carte = folium.Map(location=[lat_mean, lon_mean], zoom_start=12)

# -----------------------------
# 2️⃣ Ajouter chaque point de df_head_clean
# -----------------------------

from branca.element import Template, MacroElement

# Création du template HTML pour la légende
template = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: auto;
    height: auto;
    z-index:9999;
    font-size:14px;
    background-color:white;
    border:2px solid grey;
    border-radius:5px;
    padding: 10px;
    ">
    <b>Légende</b> <br><br>
    <i class="fa fa-map-marker fa-2x" style="color:red"></i> Maison <br>
    <i class="fa fa-map-marker fa-2x" style="color:blue"></i> Dépendance <br>
    <i class="fa fa-map-marker fa-2x" style="color:green"></i> Appartement <br>
    <i class="fa fa-map-marker fa-2x" style="color:beige"></i> Local industriel. commercial ou assimilé <br>
    <i class="fa fa-map-marker fa-2x" style="color:purple"></i> Autre
</div>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(template)
carte.get_root().add_child(macro)

# Récupérer le GeoJSON
url_geojson = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
geojson_data = requests.get(url_geojson).json()

# Ajouter les départements avec des contours
folium.GeoJson(
    geojson_data,
    name="Départements",
    style_function=lambda feature: {
        'fillColor': 'transparent',   # pas de remplissage
        'color': 'black',             # couleur du contour
        'weight': 1,                  # épaisseur du contour
    }
).add_to(carte)

geojson_data_2 = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/world_countries.json"
).json()

folium.GeoJson(geojson_data_2, name="hello world").add_to(carte)


# Optionnel : ajouter un contrôle des calques
folium.LayerControl().add_to(carte)



def typeLocal(type):
    match type:
        case "Maison":
            return folium.Icon(color="red", icon="info-sign")
        case "Dépendance":
            return folium.Icon(color="blue", icon="info-sign")
        case "Appartement":
            return folium.Icon(color="green", icon="info-sign")
        case "Local industriel. commercial ou assimilé":
            return folium.Icon(color="beige", icon="info-sign")
        case _:
            return folium.Icon(color="purple", icon="info-sign")

for idx, row in df_head_clean.iterrows():
    infos = f"Type : {row['Type local']} \nAdresse : {row['adresse_txt']} \nSurface du terrain : {row["Surface terrain"]}m² \nSurface reelle bati : {row["Surface reelle bati"]}m²"
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=infos,  # quand tu cliques sur le point, tu vois l'adresse
        icon=typeLocal(row['Type local'])
    ).add_to(carte)

# -----------------------------
# 3️⃣ Afficher la carte
# -----------------------------
carte.save("map.html")
